In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hasanmoni","key":"8da20dd5e06ff19cc2c208a892ae6eff"}'}

In [2]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission


kaggle.json


I completed this project on google colab, for this I written some code in upper cells to access
google colaboratory. If you run this project on your local machine you dont need upper cells

### Load Libraries

In [3]:
import string
import numpy as np
import PIL.Image

from os import listdir, path
from pickle import dump, load

from numpy import array
from numpy import argmax

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
from keras.callbacks import ModelCheckpoint

from nltk.translate.bleu_score import corpus_bleu


Using TensorFlow backend.


### Prepare Photo Data

In [4]:
# extract features from each photo in the directory
def extract_features(directory):
  model = VGG16()
  # re-structure the model
  model.layers.pop()
  model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
  model.summary()
  features = dict()

  for name in listdir(directory):
    filename = path.join(directory, name)
    image = load_img(filename, target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    feature = model.predict(image, verbose=0)
    image_id = name.split('.')[0]
    features[image_id] = feature
  return features



In [7]:
# extract features from all images
directory = '/content/drive/My Drive/nn/Flicker8k_Dataset'
features = extract_features(directory)
print('Extracted Features: %d' % len(features))

# save to file
dump(features, open('features.pkl', 'wb'))


Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

### Preparing Text Data

In [8]:
# loading data
def load_doc(filename):
    
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text


In [9]:
# making dictionary where key=photo_name  and value=captions 
def photo_to_description_mapping(descriptions):

    description_mapping = dict()

    # spliting text with new line
    for line in descriptions.split('\n'): 
        words = line.split()
        if len(line)<2:
            continue   

        image_id, image_description = words[0], words[1:] # seperating image name and image descriptions
        image_id = image_id.split('.')[0]
        image_description = ' '.join(image_description)
        
        #making dictionary where key=image_name and value=captions
        if image_id not in description_mapping:
            description_mapping[image_id] = list()
  
        description_mapping[image_id].append(image_description)
    
    return description_mapping


In [10]:
# cleaning data
def clean_descriptions(description_mapping):
    
    # Preapring a translation table for removing all the punctuation
    table = str.maketrans('','', string.punctuation)
    
    # Traversing through the mapping we created
    for key, descriptions in description_mapping.items():
        for i in range(len(descriptions)):
            description = descriptions[i]
            description = description.split()
            
            description = [word.lower() for word in description]
            
            description = [word.translate(table) for word in description]
            description = [word for word in description if len(word)>1]
            description = [word for word in description if word.isalpha()]
            
            descriptions[i] = ' '.join(description)


In [11]:
# Converting the loaded descriptions into a vocabulary of words

def to_vocabulary(descriptions):
    
    all_desc = set()
    
    # taking only values from every key
    for key in descriptions.keys():
        [all_desc.update(d.split()) for d in descriptions[key]]
    
    return all_desc
    

In [12]:
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list: # five descriptions of an image
            lines.append(key + ' ' + desc)
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()


In [14]:
filename = '/content/drive/My Drive/nn/Flickr8k_text/Flickr8k.token.txt'

# Loading descriptions
doc = load_doc(filename)

# Parsing descriptions
descriptions = photo_to_description_mapping(doc)
print('Loaded: %d ' % len(descriptions))

# Cleaning the descriptions
clean_descriptions(descriptions)

# Summarizing the vocabulary
vocabulary = to_vocabulary(descriptions)
print('Vocabulary Size: %d' % len(vocabulary))

# Saving to the file
save_descriptions(descriptions, 'descriptions.txt')


Loaded: 8092 
Vocabulary Size: 8763


### Loading the data

In [18]:
# Load file
def load_file(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

# Loading photos identifier
def load_photo_identifiers(filename):
    
    file = load_file(filename)
    photos = list()
    
    # Traversing the file one line at a time
    for line in file.split('\n'):
        if len(line) < 1:
            continue
        
        identifier = line.split('.')[0]
        photos.append(identifier)
    return set(photos)

# Loading clean descriptions
def load_clean_descriptions(filename, photos):
    
    file = load_file(filename)
    descriptions = dict()
    
    #traversing the file line by line
    for line in file.split('\n'):
        # splitting the line at white spaces
        words = line.split()
        image_id, image_description = words[0], words[1:]
        
        if image_id in photos:
            #creating list of description if needed
            if image_id not in descriptions:
                descriptions[image_id] = list()
            
            desc = 'startseq ' + ' '.join(image_description) + ' endseq'
            descriptions[image_id].append(desc)
            
    return descriptions


# Loading Photo features
def load_photo_features(filename, photos):
    
    #this will load the entire features
    all_features = load(open(filename, 'rb'))
    
    #we are interested in loading the features of the required photos only
    features = {k: all_features[k] for k in photos}
    
    return features
    

In [19]:
filename = '/content/drive/My Drive/nn/Flickr8k_text/Flickr_8k.trainImages.txt'

train = load_photo_identifiers(filename)
print('Dataset: ',len(train))

train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=', len(train_descriptions))

train_features = load_photo_features('features.pkl', train)
print('Photos: train=', len(train_features))


Dataset:  6000
Descriptions: train= 6000
Photos: train= 6000


### Descriptions, Tokenizer and Max length

In [20]:
# Dictionary to list of descriptions
def to_lines(descriptions):
    all_desc = list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

# Declare tokenizer
def create_tokenizer(descriptions):
    lines = to_lines(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

# calculating max length
def max_lengthTEMP(descriptions):
    lines = to_lines(descriptions)
    return max(len(d.split()) for d in lines)


In [21]:
tokenizer = create_tokenizer(train_descriptions)

vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: ', vocab_size)

max_length = max_lengthTEMP(train_descriptions)
print('Description Length: ', max_length)


Vocabulary Size:  7579
Description Length:  34


##### Defining the Model

In [22]:
#data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, tokenizer, max_length):
    while 1:
        for key, description_list in descriptions.items(): #one image and image captions at a time
            #retrieve photo features
            photo = photos[key][0]
            input_image, input_sequence, output_word = create_sequences(tokenizer, max_length, description_list, photo)
            yield [[input_image, input_sequence], output_word]
            
            
def create_sequences(tokenizer, max_length, desc_list, photo):
    X1, X2, y = list(), list(), list()
    # walk through each description for the image
    for desc in desc_list:
        # encode the sequence
        seq = tokenizer.texts_to_sequences([desc])[0]
        # split one sequence into multiple X,y pairs
        for i in range(1, len(seq)):
            # split into input and output pair
            in_seq, out_seq = seq[:i], seq[i]
            # pad input sequence
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            # encode output sequence
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
  
            X1.append(photo)
            X2.append(in_seq)
            y.append(out_seq)
    return array(X1), array(X2), array(y)

    

In [23]:
from keras.utils import plot_model
# define the captioning model
def define_model(vocab_size, max_length):
    
    # feature extractor model
    inputs1 = Input(shape=(4096,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)

    # sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)

    # decoder model
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    # summarize model
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)
    
    return model

In [24]:
model = define_model(vocab_size, max_length)
epochs = 20
steps = len(train_descriptions)
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('model_' + str(i) + '.h5')

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 34)           0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 4096)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 34, 256)      1940224     input_5[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 4096)         0           input_4[0][0]                    
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
6000/6000 [==============================] - 914s 152ms/step - loss: 4.6678
Epoch 1/1
6000/6000 [==============================] - 910s 152ms/step - loss: 3.8830
Epoch 1/1
6000/6000 [==============================] - 912s 152ms/step - loss: 3.6244
Epoch 1/1
6000/6000 [==============================] - 920s 153ms/step - loss: 3.4781
Epoch 1/1
6000/6000 [==============================] - 924s 154ms/step - loss: 3.3673
Epoch 1/1
6000/6000 [==============================] - 922s 154ms/step - loss: 3.2958
Epoch 1/1
6000/6000 [==============================] - 926s 154ms/step - loss: 3.2478
Epoch 1/1
6000/6000 [==============================] - 931s 155ms/step - loss: 3.2099
Epoch 1/1
6000/6000 [==============================] - 938s 156ms/step - loss: 3.1806
Epoch 1/1
6000/6000 [==============================] - 942s 157ms/step - loss: 3.1489
Epoch 1/1
6000/6000 [==============================] - 937s 156ms/step - loss: 3.1305
Epoch 1/1
6000/6000 [==============================] -

### Evaluate the model

In [25]:
# mapping integer to a word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

.
def generate_desc(model, tokenizer, photo, max_length):

    in_text = 'startseq'
    #iterating over the max_length since the maximum length of the description can be that only
    for i in range(max_length):
        #integer ncoding input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        #padding the input
        sequence = pad_sequences([sequence], maxlen=max_length)
        #the predict function will return probability
        prob = model.predict([photo,sequence], verbose=0)
        # probability to integer
        prob = argmax(prob)
        #calling the word_for_id function in order to map integer to word
        word = word_for_id(prob, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text

#the below function evaluates the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
    actual, predicted = list(), list()
    for key, desc_list in descriptions.items():
        prediction = generate_desc(model, tokenizer, photos[key], max_length)
        actual_desc = [d.split() for d in desc_list]
        actual.append(actual_desc)
        predicted.append(prediction.split())

    print('BLEU-1: ', corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: ', corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: ', corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: ', corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

# max length function    
def max_length(descriptions):
    lines = to_lines(descriptions)
    return max(len(d.split()) for d in lines)


In [26]:
# finding max length
max_length = max_lengthTEMP(train_descriptions)
print('Description Length: ,', max_length)

# test image processing
filename = '/content/drive/My Drive/nn/Flickr8k_text/Flickr_8k.testImages.txt'
test = load_photo_identifiers(filename)
print('Dataset: ', len(test))
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: test=', len(test_descriptions))
test_features = load_photo_features('features.pkl', test)
print('Photos: test=', len(test_features))

# calling evalute_model function
filename = '/content/model_19.h5'
model = load_model(filename)
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)


Description Length: , 34
Dataset:  1000
Descriptions: test= 1000
Photos: test= 1000


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


BLEU-1:  0.5418476686938893
BLEU-2:  0.2822494458202901
BLEU-3:  0.18403331244527216
BLEU-4:  0.07975883896257001


### Generate new descriptions

In [36]:
filename = '/content/drive/My Drive/nn/Flickr8k_text/Flickr_8k.trainImages.txt'
train = load_photo_identifiers(filename)
print('Dataset: ', len(train))
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=', len(train_descriptions))
tokenizer = create_tokenizer(train_descriptions)
dump(tokenizer, open('tokenizer.pkl', 'wb'))


Dataset:  6000
Descriptions: train= 6000


In [37]:
# feature extraction function for new image
def extract_features(filename):
    model = VGG16()
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    image = load_img(filename, target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    feature = model.predict(image, verbose=0)
    return feature

# converting number into word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

# function to generate new captions
def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = argmax(yhat)
        word = word_for_id(yhat, tokenizer) # calling function
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text


In [39]:
# loading tokenizer and model
tokenizer = load(open('tokenizer.pkl', 'rb'))
max_length = 34
model = load_model('model_19.h5')

# feature extraction from the new iamge
path = '/content/drive/My Drive/nn/example.jpg'
photo = extract_features(path)

# calling generate function to generate new caption for new image
description = generate_desc(model, tokenizer, photo, max_length)
print(description)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


startseq dog is running through the water endseq


In [40]:
# feature extracting from the new image
path = '/content/drive/My Drive/nn/hasan.png'
photo = extract_features(path)

# calling function for new caption generate from the new image
description = generate_desc(model, tokenizer, photo, max_length)
print(description)


startseq man in red shirt is standing on the sidewalk endseq


In [41]:
# feature extracting from the new image
path = '/content/drive/My Drive/nn/baby.jpg'
photo = extract_features(path)

# calling function for new caption generate
description = generate_desc(model, tokenizer, photo, max_length)
print(description)


startseq boy in blue bathing suit is swimming underwater endseq


In [42]:
# feature extracting from image
path = '/content/drive/My Drive/nn/baby play.jpeg'
photo = extract_features(path)

# calling function to generate captions
description = generate_desc(model, tokenizer, photo, max_length)
print(description)


startseq baby in red shirt is playing with red toy endseq
